### 程式 7.1 序列式 (Sequential)  v.s Keras 函數式 API：

In [ ]:
from keras.models import Sequential, Model
from keras import layers, Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))					   #1...
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64,))   #← 建立一個初始張量

# 將初始張量傳入 Dense 層得到輸出張量 x
x = layers.Dense(32, activation='relu')(input_tensor)
 
# 再將第一層的結果 x 傳入第 2 個 Dense 層得到輸出張量 y                2...
y = layers.Dense(32, activation='relu')(x) 

# 再將第二層的結果 y 傳入最後一個 Dense 層得到最後的輸出張量 output_tensor
output_tensor = layers.Dense(10, activation='softmax')(y) 

# Model 類別 "用" 初始的輸入張量和最後的輸出張量來得到模型物件
model = Model(input_tensor, output_tensor)
model.summary()     # 來看看模型摘要吧！


### 程式 7.2 以函數式 API 實作雙輸入問答模型

In [ ]:
from keras import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500
						 #↓1...                   #↓2...
text_input = Input(shape=(None, ), dtype='int32', name='text') 
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input) #← 3...
print(embedded_text.shape)  	#→ (?, ?, 64)
encoded_text = layers.LSTM(32)(embedded_text) #← 4...
print(encoded_text.shape)  #	→ (?, 32)

question_input = Input(shape=(None, ), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input) #5..
print(embedded_question.shape)  	#→ (?, ?, 32)
encoded_question = layers.LSTM(16)(embedded_question)
print(encoded_question.shape)  	#→ (?, 16)
													#↓6...
concatenated = layers.concatenate([encoded_question, encoded_text], axis=-1) 
print(concatenated.shape)  #→ (?, 48)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated) #← 7...
print(answer.shape)  #→ (?, 500) 

model = Model([text_input, question_input], answer) #← 8...
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

#1. shape = (None, ) 代表不限定張量的 shape 大小, 所以文字輸入可以是可變長度的整數序列。
#2. 請注意, 可以選擇是否為輸入命名, 原因為下面程式 7.2 中的訓練方法 2。
#3. 將輸入送進嵌入層, 編碼成大小 64 的文字嵌入向量 (處理 「參考文字」輸入)。
#4. 再透過 LSTM 層將向量序列編碼成單一個向量
#5. 處理「問題」輸入的流程 (與處理「參考文字」輸入的流程相同)
#6. 串接編碼後的「問題」和「參考文字」資料 (向量), 將兩份資料合而為一。axis 參數為 -1 代表以輸入的最後一個軸進行串接。
#7. 最後增加一個 Dense層 (softmax分類器), 將串接向量送入, 輸出模型的結果張量 answer。
#8. 在模型實例化時, 因為有兩個輸入, 所以將它們組成一個 list 一起做為輸入, 而輸出為 answer。


### 程式 7.3 將資料以兩種方式 (list、dict) 傳遞到多輸入模型進行訓練

In [ ]:
import numpy as np

num_samples = 1000
max_length = 100

# 產生 text 資料：1000 筆, 每筆 100 個字 (數字)
text = np.random.randint(1, text_vocabulary_size, 
                         size=(num_samples, max_length))
#  [  [2, 15, 8000,..... 共 100 個], [],....共 1000 筆  ]  
#      ↑   ↑    ↑         
#     產生 1 ~ 10000 (text_vocabulary_size) 區間的數字 
print(text.shape)       # (1000, 100)

# 產生 question 資料, 與上面 text 產生方式相同
question = np.random.randint(1, question_vocabulary_size, 
                             size=(num_samples, max_length))
print(question.shape)   # (1000, 100)

# 產生 answers 資料, 需為 One-hot 編碼, 共 1000 個正確答案
answers = np.random.randint(0, 1, size=(num_samples, 
                                        answer_vocabulary_size))
#  [  [0, 1, 1,..... 共 100 個], [],.... 共 1000 筆  ]
#      ↑  ↑  ↑         
#     產生 0 ~ 1 的數字 
# 此為分類器要用的 One-encoding 編碼答案    
print(answers.shape)    # (1000, 500)

# 訓練方法 1：使用 list 方式送入資料進行擬合 
#model.fit([text, question], answers, epochs=10, batch_size=128)
# 訓練方法 2：使用 dict 方式送入資料進行擬合, 鍵為 Input 層的名稱, 值為 Numpy 資料
model.fit({'text': text, 'question': question}, answers, epochs=10,  batch_size=128)


### 程式 7.4 以函數式 API 實作三個輸出結果模型

In [ ]:
from keras import layers, Input
from keras.models import Model

vocabulary_size = 50000 	#← 文章大小
num_income_groups = 10 	#← 將收入分成 10 群
                            
                          # ↓不限定輸入向量的 shape 大小
posts_input = Input(shape=(None,), dtype='int32', name='posts') 

# 用函數式 API 將輸入向量傳入 Embedding 層, 得到維度 256 的嵌入向量
embedding_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
print(embedding_posts.shape)   # ← (?, ?, 256)

# 以下以函數式 API 將嵌入向量傳入一層層之中進行處理
x = layers.Conv1D(128, 5, activation='relu')(embedding_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)  
x = layers.Dense(128, activation='relu')(x)
print(x.shape)  #← 走過一連串層之後, x.shape 為 (?, 128)

# 接下來將 x 向量分別送入 3 個輸出層。請注意, 
# 需為輸出層指定名稱(原因請見程式 7.5 中的編譯方法 2)

# 預測年紀的輸出層：純量迴歸任務
age_prediction = layers.Dense(1, name='age')(x)

# 預測收入族群的輸出層多分類任務 (10 類)
income_prediction = layers.Dense(num_income_groups, 
                                 activation='softmax', 
                                 name='income')(x)
# 預測性別的輸出層：二元分類任務
gender_prediction = layers.Dense(1, 
                                 activation='sigmoid', 
                                 name='gender')(x)

# 用輸入向量與輸出向量實例化 Model 物件
model = Model(posts_input, 
              [age_prediction, income_prediction, gender_prediction])
                 #↑ 因為輸出向量有 3 個, 所以用 list 來組成

model.summary()


### 程式 7.5 多輸出模型的編譯選項, 指定多重損失函數, 有兩種方式

In [ ]:
# 編譯方式 1 
model.compile(optimizer='rmsprop', 
              loss=['mse',		#← (需照建立層的順序)
                    'categorical_crossentropy', 
                    'binary_crossentropy'])
# 編譯方式 2 
model.compile(optimizer='rmsprop', 
              loss={'age': 'mse',	#← (需為輸出層指定名稱)
                    'income': 'categorical_crossentropy', 
                    'gender': 'binary_crossentropy'})


### 程式 7.6 孿生 (Siamese) LSTM

In [ ]:
from keras import layers
from keras import applications
from keras import Input

# 我們使用 Xception 神經網路的卷積基底 (不包含最上層的分類器) 進行影像的特徵萃取
xception_base = applications.Xception(weights=None, include_top=False)

# 建立左、右輸入張量 (左、右鏡頭影像), 其 shape 為 (250, 250, 3), 即為 250x250 的彩色影像。
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 呼叫相同的視覺模型兩次, 也就是將影像張量傳入 Xception 神經網路物件。
left_features = xception_base(left_input)
right_features = xception_base(right_input)

# 萃取出的左、右影像特徵張量 shape = (?, 8, 8, 2048)
print(left_features.shape)
print(right_features.shape)

# 串接左右影像特徵張量, shape = (?, 8, 8, 4096)
merged_features = layers.concatenate([left_features, right_features], axis=-1)
print(merged_features.shape)


### 程式 7.7 將使用 TensorBoard 的文字分類模型

In [6]:
import keras 
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))
model.summary()
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


### 程式 7.8 為 TensorBoard 紀錄檔案建立目錄 (Linux 的指令)
### $ mkdir my_log_dir

### 程式 7.9 使用 TensorBoard 回呼來訓練模型

In [ ]:
callbacks = [keras.callbacks.TensorBoard(log_dir='my_log_dir', 
                                         histogram_freq=1, 
                                         embeddings_freq=1)]

history = model.fit(x_train, y_train, 
                    epochs=20, batch_size=128, 
                    validation_split=0.2, 
                    callbacks=callbacks_list)

### 建構輕量的深度可分離卷積神經網路

In [7]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3, 
                                 activation='relu', 
                                 input_shape=(height, width, channels)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

# model.add(layers.SeparableConv2D(64, 3, activation='relu'))  # 怪怪的
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_14 (Separab (None, 62, 62, 32)        155       
_________________________________________________________________
separable_conv2d_15 (Separab (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
separable_conv2d_16 (Separab (None, 28, 28, 64)        4736      
_________________________________________________________________
separable_conv2d_17 (Separab (None, 26, 26, 128)       8896      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 128)       0         
Total params: 16,187
Trainable params: 16,187
Non-trainable params: 0
_________________________________________________________________
